In [ ]:
%load_ext autoreload
%autoreload 2

# Add project 1 to sys path 
import sys
sys.path.append('../project1/')
import tools as proj1_tools
import project2_tools as proj2_tools

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
L = 40
temps = proj2_tools.get_available_t()
states = proj2_tools.read_t(temps[0])
states.shape

In [ ]:
print(temps)
t_crit = 2.3
for t_low in temps:
    
    label = int(t_crit > t_low)
    # t_low = temps[1]
    
    plt.figure()
    states_low_t = proj2_tools.read_t(t_low)
    plt.imshow(states_low_t[1].reshape(L,L))
    plt.title(t_low)
    print(t_low)

In [ ]:
t_high = temps[-1]
states_high_t = proj2_tools.read_t(t_high)
plt.imshow(states_high_t[3].reshape(L,L))
print(t_high)

In [ ]:
def get_train_data(n, L):
    """Pick some random training data from whole data set"""
    Nt = temps.size
    Nl = L * L
    data = np.zeros((Nt, n, Nl),dtype=np.int8)
    for i,t in enumerate(temps):
        d = proj2_tools.read_t(t)
        d[np.where(d == 0)] = -1
        data[i] = d
        
    n_ordered = np.sum(temps > t_crit)
    classes = np.zeros((Nt,n),dtype = np.int8)
    classes[:-n_ordered] = 1
    
    data = data.reshape(-1, Nl)
    
    classes = classes.ravel()
    return data, classes

    
n = 10000
L = 40
data,classes = get_train_data(n, L)


In [ ]:
N_train = 1000

np.random.seed(0)
indx = np.arange(data.shape[0])
np.random.shuffle(indx)

X = data[indx][:N_train]
X = np.c_[np.ones_like(X[:,0]), X]
y = classes[indx][:N_train]


Nd = 2
Np = X.shape[1]

# Newton raphson

In [ ]:
y.shape

In [ ]:
p0

In [ ]:
p1

In [ ]:
gamma = 1e-4
beta = 2e-6*(np.random.random(size = (Np,Nd-1)) - 0.5).squeeze()

for i in range(10):
    a = (X @ beta).T
    p0  =  1/ (1 + np.exp(a)).squeeze()
    p1 = 1 - p0 #(np.exp(a)*p0).squeeze()
    
    p = np.choose(y, [p0, p1])
    
    
    Wii = np.diag(p*(1-p))
    hess = (X.T @ Wii) @ X
    deriv_cost = - X.T @ (y - p)
    
    dbeta = - np.linalg.inv(hess) @ deriv_cost
    beta = beta + gamma*dbeta
    
    correct = p > 0.5
    print(np.mean(correct))

# Gradient descent

In [ ]:
niter = 40
ngamma = 36


gamma_values = np.logspace(-8,-3,ngamma)
value = np.zeros((niter,ngamma))

colors = plt.cm.viridis(np.linspace(0,1,ngamma+1)[:-1])
print(len(colors))
for j,gamma in enumerate(gamma_values):
    beta = 1e-6*(np.random.random(size = (Np,Nd-1)) - 0.5).squeeze()
    for i in range(niter):
        #gamma = gamma_values[j]
        a = (X @ beta).T
        p0  =  1/ (1 + np.exp(a)).squeeze()
        p1 = (np.exp(a)*p0).squeeze()
        p = np.choose(y,[p0, p1])

        deriv_cost = - X.T @ (y - p)
        beta = beta - gamma * deriv_cost 

        correct = p > 0.5
        value[i,j] = np.mean(correct)
        
    if j%6 == 0 :
        plt.plot(value[:,j], c = colors[j], label='$\gamma = {:.2g}$'.format(gamma))
plt.legend()
    

In [ ]:
plt.semilogx(gamma_values, value[-1],)

In [ ]:
t_crit = 2.3
t = temps[9]
states = proj2_tools.read_t(t)

X = np.c_[np.ones(states.shape[0]), states]
plt.imshow(states[0].reshape(L,L))

In [ ]:

plt.plot(p0 + p1)

In [ ]:
classes